# Installation et démonstration de Qiskit

## 0. Installation

Installation et activation d'un environnement virtuel python:

Installer jupyter si l'un veut exécuter le code dans un notebook:

Installer paquets qiskit:

- `qiskit` paquet de base avec la plupart des fonctionnalités du language (statevectors, operators, circuits, ...) et de visualisation (`qiskit[visualization]`).

- `qiskit-ibm-runtime` pour interagir avec les resources dans le cloud (et donc aux vrais processeurs quantiques).

- `qiskit-aer` pour simuler du code efficacement sur sa propre machine. (Il apporte différente méthodes de simulation et permet d'accélérer les fausses backends en local: voir https://quantum.cloud.ibm.com/docs/en/guides/local-testing-mode)

Vérification des versions installées:

In [ ]:
from qiskit import __version__
print (__version__)

from qiskit_ibm_runtime import __version__
print (__version__)

from qiskit_aer import __version__
print (__version__)

## 1. Création d'un circuit

In [ ]:
from qiskit.circuit import QuantumCircuit

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0,1)
circuit.measure_all()

#circuit.draw()
circuit.draw(output="mpl", filename="circuit.pdf")

## 2. Choix de la platforme d'exécution

Plusieurs choix possible:
1. Réel processeur quantique
2. Faux processeur quantique (simulateur qui partage des contraintes similaires: bruit et nombre limité de qubits)
3. Simulateur AER

L'exécution peut être faite dans le cloud (par example sur les vrais QPUs) ou en local (simulateur AER). Dans le cas des faux QPUs, les deux possibilités existent. **montrer**

In [ ]:
#from qiskit_ibm_runtime import QiskitRuntimeService
#service = QiskitRuntimeService(channel="ibm_quantum_platform", token="DL3Y9nQgo30ZGzyfv5Vo36spm7TaK2bsLITzOH9CGSff")

#backend = service.least_busy(simulator=False, operational=True)
#backend = service.least_busy(simulator=True, operational=True)

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

backend = FakeManilaV2()

In [ ]:
#from qiskit_aer import AerSimulator

#backend = AerSimulator()

## 3. La "compilation"

Il s'agit ensuite d'adapter et optimiser le circuit pour la platforme choisie. Deux interfaces:

1. `qiskit.compiler` offre une interface simple, un peu comme un compilateur classique
2. `qiskit.transpiler` permet de personaliser les étapes de transpilation

In [ ]:
from qiskit.compiler import transpile
circuit_transpiled = transpile(circuit, backend, optimization_level=3)

#from qiskit.transpiler import generate_preset_pass_manager
#pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
#circuit_transpiled = pm.run(circuit)

print("Transpiled for backend =", backend.name)
circuit_transpiled.draw(output="mpl", filename="circuit_transpiled.pdf")

## 4. Exécution

**TODO:** Décrire la procédure quand on interagit avec les vrais QPU. De manière générale, montrer comment récupérer les jobs en différé lorsque l'on utilise les ressources cloud.


In [ ]:
from qiskit_ibm_runtime import SamplerV2 as Sampler
sampler = Sampler(backend)
job = sampler.run([circuit_transpiled], shots=1024)

job_id = job.job_id()
print("Job id: ", job_id)

## 5. Analyse des résultats

In [ ]:
#job = service.job(job_id)
result = job.result()

from qiskit.visualization import plot_histogram
plot_histogram(result[0].data.meas.get_counts())

- Avec le simulateur AER, on ne mesurera jamais la paire $\ket{01}$ ni $\ket{10}$.
- Les QPU actuels sont quant à eux susceptibles à un fort taux d'erreur: l'état quantique est progressivement éparpillé sur l'ensemble des états de la base computationnelle, ce qui amène à des résultats inattendus lors de la mesure.